### Import Required Packages

In [ ]:
# Imports
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from random import randint #for random colour generation
pd.set_option('display.float_format', lambda x: '%.5f' % x)
from IPython.display import display, HTML

### Input Data From User 

In [ ]:
#Market analysed: 'Investment','FullYear','DayAhead','Balancing' (choose one or several)
markets=['FullYear'] 
input_price = 'PriceElectricityHourly'

regions = ['BE', 'NL', 'DK1'] # Choose list of regions (e.g. ['BE', 'NL', 'DK1'] or 'all' (i.e. all regions that produce on-shore wind) )

color = ['#06266F',  '#a64b00', '#006363', '#a66f00']

### Plot Settings

In [ ]:
# Set plotting specifications
%matplotlib inline
plt.rcParams.update({'font.size': 21})
plt.rcParams['xtick.major.pad']='12'
plt.rc('legend', fontsize=16)
y_limit = 1.1
lw = 3

# Read Input Files

#### Price and Wind Production

In [ ]:
price_data=pd.DataFrame()
for market in markets:
    csvfiles = []
    for file in glob.glob("./input/results/" + market + "/*.csv"):
        csvfiles.append(file)

    csvfiles=[file.replace('./input\\','') for file in csvfiles] 
    csvfiles=[file.replace('.csv','') for file in csvfiles]  
    csvfiles=[file.split('_') for file in csvfiles]  
    csvfiles = np.asarray(csvfiles)  
    csvfiles=pd.DataFrame.from_records(csvfiles)
    
    csvfiles.rename(columns={0: 'Output', 1: 'Scenario',2: 'Year',3:'Subset'}, inplace=True)
    scenarios=csvfiles.Scenario.unique().tolist()
    years=csvfiles.Year.unique().tolist()
    subsets=csvfiles.Subset.unique().tolist()

    for scenario in scenarios:
        for year in years:
            for subset in subsets:
                price_file = "./input/results/"+ market + "/" + input_price + "_" + scenario + "_" + year + "_" + subset + ".csv"
                if os.path.isfile(price_file):
                    df=pd.read_csv(price_file,encoding='utf8')
                    df['Scenario'] = scenario
                    df['Market']   = market
                    df.rename(columns = {'Val' : 'Price'}, inplace = True)
                    price_data=price_data.append(df)

#### Replace possible "Eps" with 0

In [ ]:
price_data.Price=price_data.Price.replace('Eps', 0)
price_data.Price=pd.to_numeric(price_data.Price)

In [ ]:
price_data = price_data.sort_values(by='Price', ascending = False)
price_data['exceedence'] = np.array(np.arange(1., len(price_data.Price)+1) / len(price_data.Price) )


In [ ]:
fig = px.line(price_data,x = 'exceedence',y= 'Price', labels = {'exceedence': "Exceedence (fraction of time)", 'Price': "Price (€ / MWh)"}, \
                 title = 'Price Duration Curve ' + scenario + ', ' + year)
fig.show()